# Кибериммунная автономность$\\$Создание конструктивно защищённого автономного наземного транспортного средства$\\$Модуль 4

## О документе

Версия 1.03

Модуль 4 для регионального этапа соревнований по кибериммунной автономности

### Модуль 4. Следование по трассе с киберпрепятствиями

Для успешного выполнения этого задания необходимое активировать специальный режим работы виртуальной машинки - в рамках этого задания можно менять только блоки, отвечающие за безопасность (ограничитель, монитор безопасности), **другие блоки менять запрещено**.

При прохождении маршрута будут имитироваться атаки со стороны злоумышленников, которые будут пытаться нарушить цели безопасности. Важно, чтобы им это не удалось.

1. Запустите свою машинку и убедитесь, что она проходит всю трассу без нарушений ограничений скорости. При необходимости измените логику работы блока безопасности
2. Добавьте контроль доставки груза в модуле SafetyBlock - убедитесь, что груз доставляется до конечной точки маршрута.
3. Если в модуле 3 вы реализовали монитор безопасности - не забудьте его перенести в этот модуль, это принесёт дополнительные баллы!

Активация киберпрепятствий в системе управления:

после инициализации системы управления добавьте следующую строку
```python
control_system.enable_surprises()
```

В этом блоке добавьте все ваши реализации изменённых бортовых систем

In [1]:
# ваш код
from geopy import Point as GeoPoint
from src.security_monitory import BaseSecurityMonitor
from src.security_policy_type import SecurityPolicy
from multiprocessing import Queue
from src.event_types import Event
from src.navigation_system import BaseNavigationSystem
from src.communication_gateway import BaseCommunicationGateway
from src.control_system import BaseControlSystem
from src.safety_block import BaseSafetyBlock
from src.config import COMMUNICATION_GATEWAY_QUEUE_NAME, NAVIGATION_QUEUE_NAME, SAFETY_BLOCK_QUEUE_NAME, CONTROL_SYSTEM_QUEUE_NAME, SECURITY_MONITOR_QUEUE_NAME, SERVOS_QUEUE_NAME, CARGO_BAY_QUEUE_NAME
from src.config import LOG_DEBUG, LOG_ERROR, LOG_INFO
import math

    
class SecurityMonitor(BaseSecurityMonitor):
    """ класс монитора безопасности """

    def __init__(self, queues_dir):
        super().__init__(queues_dir)
        self._init_set_security_policies()

    def _init_set_security_policies(self):
        """ инициализация политик безопасности """
        default_policies = [
            SecurityPolicy(
                source=COMMUNICATION_GATEWAY_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation='set_mission'
            ),
            SecurityPolicy(
                source=COMMUNICATION_GATEWAY_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_mission'
            ),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_speed'
            ),
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=SERVOS_QUEUE_NAME,
                operation='set_speed'
            ),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_direction'
            ),
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=SERVOS_QUEUE_NAME,
                operation='set_direction'
            ),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='release_cargo'
            ),
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=CARGO_BAY_QUEUE_NAME,
                operation='release_cargo'
            ),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='lock_cargo'
            ),
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=CARGO_BAY_QUEUE_NAME,
                operation='lock_cargo'
            ),
            SecurityPolicy(
                source=NAVIGATION_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='position_update'
            ),
            SecurityPolicy(
                source=NAVIGATION_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation='position_update'
            ),
        ]
        self.set_security_policies(policies=default_policies)        

    def set_security_policies(self, policies):
        """ установка новых политик безопасности """
        self._security_policies = policies
        self._log_message(LOG_INFO, f"изменение политик безопасности: {policies}")

    def _check_event(self, event: Event):
        """ проверка входящих событий """
        self._log_message(LOG_DEBUG, f"проверка события {event}, по умолчанию выполнение запрещено")

        authorized = False
        request = SecurityPolicy(
            source=event.source,
            destination=event.destination,
            operation=event.operation)

        if request in self._security_policies:
            self._log_message(LOG_DEBUG, "событие разрешено политиками, выполняем")
            authorized = True

        if authorized is False:
            self._log_message(LOG_ERROR, f"событие не разрешено политиками безопасности! {event}")
        return authorized

# 2. Модифицируем блоки для отправки через SecurityMonitor

class CommunicationGateway(BaseCommunicationGateway):
    def _send_mission_to_consumers(self):
        event = Event(
            source=self.event_source_name,
            destination=CONTROL_SYSTEM_QUEUE_NAME,
            operation="set_mission",
            parameters=self._mission
        )
        security_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_q.put(event)
        event = Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="set_mission",
            parameters=self._mission
        )
        security_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_q.put(event)

class ControlSystem(BaseControlSystem):
    def _send_speed_and_direction_to_consumers(self, speed, direction):
        events = [
            Event(
                source=BaseControlSystem.event_source_name,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation="set_speed",
                parameters=speed
            ),
            Event(
                source=BaseControlSystem.event_source_name,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation="set_direction",
                parameters=direction
            )
        ]
        security_q = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        for event in events:
            security_q.put(event)

    def _release_cargo(self):
        safety_q_name = SAFETY_BLOCK_QUEUE_NAME

        event_release_cargo = Event(
            source=BaseControlSystem.event_source_name,
            destination=safety_q_name,
            operation="release_cargo",
            parameters=None
        )

        safety_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        safety_q.put(event_release_cargo)

    def _lock_cargo(self):
        safety_q_name = SAFETY_BLOCK_QUEUE_NAME

        event_lock_cargo = Event(
            source=BaseControlSystem.event_source_name,
            destination=safety_q_name,
            operation="lock_cargo",
            parameters=None
        )

        safety_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        safety_q.put(event_lock_cargo)       

class NavigationSystem(BaseNavigationSystem):
    def _send_position_to_consumers(self):
        event = Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="position_update",
            parameters=self._position
        )
        security_q = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_q.put(event)
        event = Event(
            source=self.event_source_name,
            destination=CONTROL_SYSTEM_QUEUE_NAME,
            operation="position_update",
            parameters=self._position
        )
        security_q = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_q.put(event)

class SafetyBlock(BaseSafetyBlock):
    """ класс ограничений безопасности """

    def _set_new_direction(self, direction: float):
        """ установка нового направления перемещения """

        self._log_message(LOG_INFO, f"текущие координаты: {self._position}")
        self._log_message(LOG_DEBUG, f"маршрутное задание: {self._mission}")
        self._log_message(LOG_DEBUG, f"состояние маршруте: {self._route}")
        # TODO реализовать контроль безопасности изменения направления
        target_point = self._route.next_point()
        current_point = self._position
        target_direction = self._calculate_bearing(current_point, target_point)
        if not self._mission or not self._position or not self._route:
            self._log_message(LOG_ERROR, f"Неизвестный маршрут или позиция! Остановка.")
            self._direction = 0
            self._speed = 0
            self._send_direction_to_consumers()
            self._send_speed_to_consumers()
            return

        if abs(direction - target_direction) > 1:  # Допустимое отклонение 1 градус
            self._log_message(LOG_ERROR, f"Некорректное направление! Запрошено: {direction}, ожидалось: {target_direction}")
            self._direction = target_direction
            self._send_direction_to_consumers()
            return  
        
        self._direction = direction
        self._send_direction_to_consumers()

    def _set_new_speed(self, speed: float):
        """ установка новой скорости """
        # TODO реализовать контроль безопасности изменения скорости
        current_limit = self._route.calculate_speed()

        if not self._mission or not self._position or not self._route:
            self._log_message(LOG_ERROR, f"Неизвестный маршрут или позиция! Остановка.")
            self._direction = 0
            self._speed = 0
            self._send_direction_to_consumers()
            self._send_speed_to_consumers()
            return
        
        if speed > current_limit:
            self._log_message(LOG_ERROR, f"Превышение скорости! Лимит: {current_limit}, запрошено: {speed}")
            self._speed = current_limit
        else:
            self._speed = speed
        
        self._send_speed_to_consumers()
        self._send_direction_to_consumers()
    
    def _calculate_bearing(self, start: GeoPoint, end: GeoPoint) -> float:
        delta_longitude = end.longitude - start.longitude
        x = math.sin(math.radians(delta_longitude)) * \
            math.cos(math.radians(end.latitude))
        y = math.cos(math.radians(start.latitude)) * math.sin(math.radians(end.latitude)) - \
            math.sin(math.radians(start.latitude)) * math.cos(math.radians(end.latitude)) * \
            math.cos(math.radians(delta_longitude))

        initial_bearing_rad = math.atan2(x, y)

        initial_bearing_deg = math.degrees(initial_bearing_rad)

        compass_bearing = (initial_bearing_deg + 360) % 360

        return compass_bearing


    def _send_speed_to_consumers(self):
        self._log_message(LOG_DEBUG, "отправляем скорость получателям")
        servos_q_name = SERVOS_QUEUE_NAME
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # отправка сообщения с желаемой скоростью
        event_speed = Event(source=self.event_source_name,
                            destination=servos_q_name,
                            operation="set_speed",
                            parameters=self._speed
                            )
        security_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_q.put(event_speed)

    def _send_direction_to_consumers(self):
        self._log_message(LOG_DEBUG, "отправляем направление получателям")

        servos_q_name = SERVOS_QUEUE_NAME
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # отправка сообщения с желаемой скоростью
        event_direction = Event(source=self.event_source_name,
                            destination=servos_q_name,
                            operation="set_direction",
                            parameters=self._direction
                            )
        security_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME) 
        security_q.put(event_direction)

    def _lock_cargo(self, _):
        self._log_message(LOG_INFO, "Блокировка грузового отсека")
        self._send_lock_cargo_to_consumers()

    def _release_cargo(self, _):
        if not self._mission or not self._position or not self._route:
            self._log_message(LOG_ERROR, "Неизвестный маршрут!")
            self._speed = 0
            self._direction = 0
            self._send_speed_to_consumers()
            self._send_direction_to_consumers()
            return
        if not self._route.route_finished:
            self._log_message(LOG_INFO, "Выгружать еще рано! ")
            return
        self._send_release_cargo_to_consumers()
        
    def _send_lock_cargo_to_consumers(self):
        self._log_message(LOG_DEBUG, "Отправляем команду на блокировку грузового отсека")
        event = Event(
            source=self.event_source_name,
            destination=CARGO_BAY_QUEUE_NAME,
            operation="lock_cargo",
            parameters=None
        )
        security_q : Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_q.put(event)

    def _send_release_cargo_to_consumers(self):
        self._log_message(LOG_DEBUG, "Отправляем команду на блокировку грузового отсека")
        event = Event(
            source=self.event_source_name,
            destination=CARGO_BAY_QUEUE_NAME,
            operation="release_cargo",
            parameters=None
        )
        security_q : Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_q.put(event)

Если у вас настроена и работает СУПА, установите в True значение переменной afcs_present

In [2]:
afcs_present = True

Поменяем идентификатор машинки для этого модуля

In [3]:
car_id = "m4"

В следующем блоке измените маршрут на ваш, его можно скопировать из модуля 2.
 
Для проверки работы систем безопасности будут активированы киберпрепятствия

```python
control_system.enable_critical_surprises()
```

In [ ]:
# используем то же маршрутное задание, которое было в модуле 2
from time import sleep

from src.queues_dir import QueuesDirectory
from src.servos import Servos
from src.sitl import SITL
from src.cargo_bay import CargoBay
from src.mission_planner import MissionPlanner
from src.config import LOG_ERROR, LOG_INFO
from src.mission_planner_mqtt import MissionSender
from src.mission_planner import Mission
from src.sitl_mqtt import TelemetrySender
from src.system_wrapper import SystemComponentsContainer
from src.wpl_parser import WPLParser
from src.mission_type import GeoSpecificSpeedLimit


wpl_file = "kupchino_route.wpl"

parser = WPLParser(wpl_file)    
points = parser.parse()

# обновите скоростные ограничения для вашего маршрута!S
speed_limits = [
    GeoSpecificSpeedLimit(0, 60),
    GeoSpecificSpeedLimit(19, 100), 
    GeoSpecificSpeedLimit(26, 60)
]


home = points[0]
mission = Mission(home=home, waypoints=points,speed_limits=speed_limits, armed=True)

# каталог очередей для передачи сообщений между блоками
queues_dir = QueuesDirectory() 


# создание блоков передачи данных в СУПА
if afcs_present:
    mission_sender = MissionSender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)
    telemetry_sender = TelemetrySender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)

# создание основных функциональных блоков
mission_planner = MissionPlanner(
    queues_dir, afcs_present=afcs_present, mission=mission)

sitl = SITL(
    queues_dir=queues_dir, position=home,
    car_id=car_id, post_telemetry=afcs_present, log_level=LOG_ERROR)

communication_gateway = CommunicationGateway(
    queues_dir=queues_dir, log_level=LOG_ERROR)
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)

navigation_system = NavigationSystem(
    queues_dir=queues_dir, log_level=LOG_ERROR)

servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)

safety_block = SafetyBlock(queues_dir=queues_dir, log_level=LOG_INFO)

security_monitor = SecurityMonitor(queues_dir=queues_dir)
# сборка всех запускаемых блоков в одном "кузове"
system_components = SystemComponentsContainer(
    components=[
        mission_sender,
        telemetry_sender,
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        safety_block,
        security_monitor
    ] if afcs_present else [
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        safety_block,
        security_monitor
    ])

#################################
# АКТИВАЦИЯ КИБЕРПРЕПЯТСТВИЙ
control_system.enable_surprises()
#################################

# запуск всех блоков
system_components.start()

# ограничение поездки по времени
# в случае превышения времени выполнения ячейки на более чем 10 секунд от заданного, 
# допустимо перезапустить вычислительное ядро и повторно выполнить весь блокнот, штрафные очки за это не начисляются
# при условии, что повторный запуск закончился успешно
sleep(630)

# останавливаем все компоненты
system_components.stop()

# удалим все созданные компоненты
system_components.clean()

[ИНФО][QUEUES] создан каталог очередей
[ИНФО][QUEUES] регистрируем очередь planner.mqtt
[ИНФО][QUEUES] регистрируем очередь sitl.mqtt
[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий
[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][QUEUES] регистрируем очередь cargo
[ИНФО][CARGO] создан компонент грузового отсека, отсек заблокирован
[ИНФО][QUEUES] регистрируем очередь safety
[ИНФО][SAFETY] создан ограничитель
[ИНФО][QUEUES] регистрируем очередь security
[ИНФО][SECURITY] создан монитор безопасности
[ИНФО][SECURITY] изменение политик безопасности: [SecurityPolicy(source='communication', destination='control', operation='set_mission'), SecurityPolicy(source='communication', destination='safety', operation='set_m

Убедитесь, что 
1. ваша машинка успешно прошла весь заданный маршрут
2. не превысила ограничения скорости
3. успешно доставила груз

Если всё так - поздравляем, вы справились с заданием! Обязательно зафиксируйте все изменения в репозитории!



Мы будем признательны за обратную связь - любые комментарии, которые вы можете дать по итогам выполнения этого задания. 

Например, 

- было ли задание понятным по шкале 1..10 (1 - ничего не понятно, 10 - вопросов вообще не было, всё понятно)?
- было ли задание интересным по шкале 1..10 (1 - скука смертная, 10 - лучшее, что вам пока встречалось на олимпиадах)? 
- что бы вы предложили изменить, чтобы сделает его более интересным?
- по шкале 1..10 насколько сложным оно было для вас?
- что было самым трудным в задании? 

Авторы наиболее развёрнутых и интересных комментариев получат особенный приз от Лаборатории Касперского!

Дополнительная информация о кибериммунной разработке
- https://os.kaspersky.ru/cyber-immune-development/ 
- https://github.com/sergey-sobolev/cyberimmune-systems/wiki/%D0%9A%D0%B8%D0%B1%D0%B5%D1%80%D0%B8%D0%BC%D0%BC%D1%83%D0%BD%D0%B8%D1%82%D0%B5%D1%82
- канал в телеграм: https://t.me/learning_cyberimmunity
